<a href="https://colab.research.google.com/github/bhattacharjee/mtu-nlp-assignment/blob/main/web-examples/bert_qa_rahul_agarwal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch
!pip install transformers


     |████████████████████████████████| 3.4 MB 15.6 MB/s 
     |████████████████████████████████| 895 kB 59.5 MB/s 
     |████████████████████████████████| 596 kB 60.0 MB/s 
     |████████████████████████████████| 3.3 MB 17.4 MB/s 
     |████████████████████████████████| 61 kB 662 kB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [19]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
#from datasets import load_dataset, load_metric
import random
from transformers import AutoTokenizer
import transformers
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch
from transformers import default_data_collator
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
import torch

cuda = torch.device('cuda')
torch.set_default_tensor_type('torch.cuda.FloatTensor')
bert_model_name = "bert-large-uncased-whole-word-masking-finetuned-squad"
tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
model = AutoModelForQuestionAnswering.from_pretrained(bert_model_name)


In [22]:
import time
text = r"""
🤗 Transformers (formerly known as pytorch-transformers and pytorch-pretrained-bert) provides general-purpose
architectures (BERT, GPT-2, RoBERTa, XLM, DistilBert, XLNet…) for Natural Language Understanding (NLU) and Natural
Language Generation (NLG) with over 32+ pretrained models in 100+ languages and deep interoperability between
TensorFlow 2.0 and PyTorch.
"""
questions = [
    "How many pretrained models are available in Transformers?",
    "What does Transformers provide?",
    "Transformers provides interoperability between which frameworks?",
]
for question in questions:
    t1 = time.monotonic()
    inputs = tokenizer.encode_plus(\
                question,\
                text,\
                add_special_tokens=True,\
                return_tensors="pt")

    input_ids = inputs["input_ids"].tolist()[0]
    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)

    pred = model(**inputs)
    answer_start_scores = pred['start_logits'][0]
    answer_end_scores = pred['end_logits'][0]

    # Get the most likely beginning of answer with the argmax of the score
    answer_start = torch.argmax(answer_start_scores)
    # Get the most likely end of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1

    print(answer_start_scores.detach().cpu().numpy().shape)
    print(answer_end_scores.detach().cpu().numpy().shape)
    print(answer_start, answer_end)

    print("--->\n", inputs['input_ids'].detach().cpu().numpy())


    answer = tokenizer\
                    .convert_tokens_to_string(\
                        tokenizer.convert_ids_to_tokens(\
                            input_ids[answer_start:answer_end]))
    print(f"Question: {question}")
    print(f"Answer: {answer}\n")
    print(f"Time taken = {time.monotonic() - t1}")

(110,)
(110,)
tensor(81) tensor(84)
--->
 [[  101  2129  2116  3653 23654  2098  4275  2024  2800  1999 19081  1029
    102   100 19081  1006  3839  2124  2004  1052 22123  2953  2818  1011
  19081  1998  1052 22123  2953  2818  1011  3653 23654  2098  1011 14324
   1007  3640  2236  1011  3800  4294  2015  1006 14324  1010 14246  2102
   1011  1016  1010 23455  1010 28712  2213  1010  4487 16643 23373  1010
  28712  7159  1529  1007  2005  3019  2653  4824  1006 17953  2226  1007
   1998  3019  2653  4245  1006 17953  2290  1007  2007  2058  3590  1009
   3653 23654  2098  4275  1999  2531  1009  4155  1998  2784  6970 25918
   8010  2090 23435 12314  1016  1012  1014  1998  1052 22123  2953  2818
   1012   102]]
Question: How many pretrained models are available in Transformers?
Answer: over 32 +

Time taken = 0.054235806999940905
(104,)
(104,)
tensor(32) tensor(37)
--->
 [[  101  2054  2515 19081  3073  1029   102   100 19081  1006  3839  2124
   2004  1052 22123  2953  2818  1011 1

In [6]:
!nvidia-smi

Fri Dec 17 19:33:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------